In [1]:
from myo import *
import datetime
import math
import logging
import argparse
import time
import os
import shutil
import sys
import pyaudio
import wave
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# from IPython.display import clear_output
from threading import Thread
from multiprocessing import Process
from os import listdir
from pythonosc import osc_message_builder
from pythonosc import udp_client

PARTICIPANT = "william"
WAVE_OUTPUT_FILENAME = 'nm_test1'
FORMAT = pyaudio.paInt16
CHANNELS = 1
DEVICE_INDEX = 3
RATE = 44100
CHUNK = 64
RECORD_SECONDS = 5
BPM = 70
PORT = 3000 #57120 


TTY = [f for f in listdir("/dev") if f.startswith("tty.usbmodem")]
print(TTY)

MAC = []
try:
    for tty in TTY: # search every usb adapter
        mac = Myo(adapter=BT(tty="/dev/"+tty, baudrate=115200)).find_mac()
        print("In tty {} we found {}".format(tty, mac))
        for m in mac:
            if m not in MAC:
                MAC.append(m)
except:
    # Hard-coding
    MAC = ["d0:8d:fc:7f:f5:f1", "d9:db:d4:2b:d4:17"]
#     MAC = ['d0:8d:fc:7f:f5:f1', 'ef:9d:fd:31:ea:10'] 
#     MAC = ["c3:0c:f3:14:d7:f0", "ef:9d:fd:31:ea:10"]


# MAC = ['d0:8d:fc:7f:f5:f1', 'ef:9d:fd:31:ea:10']

print(MAC)
print("\n\n\nIf the the number is not correct, try to adjust Myo connection and do it again.\n\n\n")


#define MYO_ADDRESS  "cd:2a:24:45:66:3f" // SVERM 1
#define MYO_ADDRESS  "fb:61:d1:12:3e:77" // SVERM 2
#define MYO_ADDRESS  "c8:2f:84:e5:88:af" // SVERM 3
#define MYO_ADDRESS  "f4:0f:df:81:1e:1b" // SVERM 4
#define MYO_ADDRESS  "c9:50:e1:71:71:81" // SVERM 5
#define MYO_ADDRESS  "dd:f0:45:ba:5a:ad" // SVERM 6
#define MYO_ADDRESS  “d0:8d:fc:7f:f5:f1” // SVERM 7
#define MYO_ADDRESS  “ef:9d:fd:31:ea:10” // SVERM 8
#define MYO_ADDRESS  “c4:ca:8f:db:03:6b” // SVERM 9
#define MYO_ADDRESS  “d4:61:dc:9e:4d:cf” // SVERM 10


myo = []
osc_client = udp_client.SimpleUDPClient("localhost", PORT)

osc_client.send_message("/taskname", WAVE_OUTPUT_FILENAME)

#osc_client.send_message("/task", WAVE_OUTPUT_FILENAME.split('_')[0])
#osc_client.send_message("/dynamic", WAVE_OUTPUT_FILENAME.split('_')[-2])


data_dir = os.path.join(os.getcwd(), 'data')
newdir = os.path.join(data_dir, PARTICIPANT, WAVE_OUTPUT_FILENAME)

if not os.path.exists(os.path.join(data_dir, PARTICIPANT)):
    os.mkdir(os.path.join(data_dir, PARTICIPANT))
if not os.path.exists(newdir):
    os.mkdir(newdir)

['tty.usbmodem11']
Discovering available Myos...
Found a Myo: SVERM7 MAC: d0:8d:fc:7f:f5:f1
In tty tty.usbmodem11 we found ['d0:8d:fc:7f:f5:f1']
['d0:8d:fc:7f:f5:f1']



If the the number is not correct, try to adjust Myo connection and do it again.





In [2]:
def proc_emg(index):
    def handler(emg_data):
        address = "/emg"+str(index)
        osc_client.send_message(address, emg_data)
        
    return handler # should return the function so the *index can be identical

def proc_imu(index):
    def handler(quat_data, acc_data, gyro_data):
#         address = "/acc"+str(index)
#         osc_client.send_message(address, acc_data)
        address = "/acc" + str(index)
        address2 = "/quat" + str(index)
        address3 = "/gyro" + str(index)
        osc_client.send_message(address, acc_data)
        osc_client.send_message(address2, quat_data)
        osc_client.send_message(address3, gyro_data)
    return handler

def init_myo(index, time):
    m = Myo(adapter=BT(tty="/dev/"+TTY[index], baudrate=115200), start_time=time)

    m.add_emg_handler(proc_emg(index))   #comment out these 2 lines when not using osc
    m.add_imu_handler(proc_imu(index))

    m.connect(address=MAC[index])
    m.sleep_mode(1)
    m.set_mode(EMG_Mode.send_emg.value, IMU_Mode.send_data.value, Classifier_Mode.disabled.value)
    m.vibrate(1)
    myo.append(m)

def run_myo(index):
    try:
        print("myo {} starts at: {}".format(index, time.time()-TIME) )
        while True:
            myo[index].run()
    except KeyboardInterrupt:
        myo[index].emg_to_csv()
        myo[index].imu_to_csv()
        myo[index].disconnect()
        print("\nDisconnected")
    finally:
        pass

def record():

    Recordframes = []
    audio = pyaudio.PyAudio()
    index = DEVICE_INDEX
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,input_device_index = index,
                frames_per_buffer=CHUNK)
    
    middle_time = time.time()
    audio_start = middle_time - TIME

    print ("Audio recording starts at: ", audio_start)
    try:        
        while True:
            data = stream.read(CHUNK)        
            Recordframes.append(data)
    except KeyboardInterrupt:
        pass
    finally:
        print ("recording stopped")
        stream.stop_stream()
        stream.close()
        audio.terminate()
        waveFile = wave.open('./data/'+WAVE_OUTPUT_FILENAME+'.wav', 'wb')
        waveFile.setnchannels(CHANNELS)
        waveFile.setsampwidth(audio.get_sample_size(FORMAT))
        waveFile.setframerate(RATE)
        waveFile.writeframes(b''.join(Recordframes))
        waveFile.close()
        
        file=open('./data/audio_start.txt','w')
        file.write(str(audio_start))
        file.close()
        
        print("total proccessing time:", time.time() - middle_time )
        
def metro():
    timeline = []
    beat = 0
    try:
        while True:
            osc_client.send_message("/metro", beat)
            t = time.time()
            timeline.append(t-TIME)
            if beat < 3:
                beat += 1
            else:
                beat = 0
            time.sleep(60/BPM)
    except KeyboardInterrupt:
        pass
    finally:
        file=open('./data/timeline.txt','w')
        file.write( str(timeline).replace('[','').replace(']','') )
        file.close()
        #osc_client.send_message("/reset", 1)


# MAC = ['d0:8d:fc:7f:f5:f1', 'ef:9d:fd:31:ea:10']
# MAC = ["d9:db:d4:2b:d4:17", "d0:8d:fc:7f:f5:f1"]

# Myo-1 MAC: c3:0c:f3:14:d7:f0

# MAC = ['ef:9d:fd:31:ea:10', 'd0:8d:fc:7f:f5:f1']

# MAC = ["c3:0c:f3:14:d7:f0", "ef:9d:fd:31:ea:10"]

# MAC = [MAC[1], MAC[0]]

TIME = time.time()
init_myo(0, TIME)
# init_myo(1, TIME)

# while True:
#     init_1 = Process(target=init_myo, args=(0, TIME, False))
#     init_2 = Process(target=init_myo, args=(1, TIME, False))
#     init_1.start()
#     init_2.start()
#     init_1.join(timeout=3)
#     init_2.join(timeout=3)
#     if (init_1.exitcoed == 0) & (init_2.exitcoed == 0):
#         break
#     else:
#         init_1.terminate()
#         init_2.terminate()

Connecting to Myo: d0:8d:fc:7f:f5:f1
Myo Connected.
name: SVERM7
firmware: 1.5.1970.2


In [3]:
myo_1 = Process(target=run_myo, args=(0,))
# myo_2 = Process(target=run_myo, args=(1,))
rec = Process(target=record, args=())
metro = Process(target=metro, args=())

try:
    myo_1.start()
#     myo_2.start()
    rec.start()
    metro.start()
    
    metro.join()
    rec.join()
#     myo_2.join()
    myo_1.join()
except KeyboardInterrupt:
    osc_client.send_message("/reset", 0)
    time.sleep(5)
    for f in os.listdir(data_dir):
        suffix = '.txt', '.csv', '.wav'
        if f.endswith(suffix):
            shutil.move(os.path.join(data_dir, f), 
                        os.path.join(newdir, f))

myo 0 starts at: 2.3608601093292236

Disconnected


In [ ]:
31 * 8 * 6 * 4